In [1]:
import pandas as pd

from bokeh.plotting import figure, output_notebook, show
output_notebook()
from bokeh.models.sources import ColumnDataSource as cds

Loading BokehJS ...

In [3]:
df_teamsplits = pd.read_pickle('pickle/teamsplits_periods.pickle')
df_scoring = pd.read_pickle('pickle/scoring.pickle')

In [6]:
ducks = df_teamsplits.loc[df_teamsplits["name"] == "Anaheim Ducks", :]
ducks = ducks.sort_index()
ducks.head()

,name,L,OL,T,W
month,,,,,
2006-01,Anaheim Ducks,5.0,2.0,NaN,4.0
2006-02,Anaheim Ducks,5.0,2.0,NaN,5.0
2006-03,Anaheim Ducks,3.0,2.0,NaN,10.0
2006-04,Anaheim Ducks,0.0,2.0,NaN,1.0
2006-10,Anaheim Ducks,0.0,3.0,NaN,9.0


In [7]:
ducks.resample("A").sum()

,L,OL,T,W
month,,,,
2006,20.0,14.0,0.0,48.0
2007,27.0,8.0,0.0,47.0
2008,33.0,7.0,0.0,42.0
2009,32.0,11.0,0.0,39.0
2010,30.0,5.0,0.0,47.0
2011,36.0,12.0,0.0,34.0


In [8]:
ducks.index = ducks.to_timestamp().index.map(
    lambda x: 
    x + pd.DateOffset(years=1) if x.month < 5
    else x
)
ducks = ducks.sort_index()
ducks.head()

,name,L,OL,T,W
month,,,,,
2006-10-01,Anaheim Ducks,0.0,3.0,NaN,9.0
2006-11-01,Anaheim Ducks,2.0,3.0,NaN,10.0
2006-12-01,Anaheim Ducks,5.0,0.0,NaN,9.0
2007-01-01,Anaheim Ducks,5.0,2.0,NaN,4.0
2007-02-01,Anaheim Ducks,5.0,2.0,NaN,5.0


In [10]:
resampler = ducks.resample("A-JUN")

In [12]:
res = resampler.apply(
        lambda x: x.reset_index()[["W", "L"]]
                   .expanding().sum()
)
res.head(10)

W     L
month                   
2007-06-30 0   9.0   0.0
           1  19.0   2.0
           2  28.0   7.0
           3  32.0  12.0
           4  37.0  17.0
           5  47.0  20.0
           6  48.0  20.0
2008-06-30 0   1.0   1.0
           1   4.0   7.0
           2  12.0  11.0

In [13]:
res.index = ducks.index
res.head()

,W,L
month,,
2006-10-01,9.0,0.0
2006-11-01,19.0,2.0
2006-12-01,28.0,7.0
2007-01-01,32.0,12.0
2007-02-01,37.0,17.0


In [15]:
final = res.asfreq(pd.tseries.offsets.MonthBegin())
final.head()

,W,L
month,,
2006-10-01,9.0,0.0
2006-11-01,19.0,2.0
2006-12-01,28.0,7.0
2007-01-01,32.0,12.0
2007-02-01,37.0,17.0


In [16]:
fdf = final.reset_index()
fdf.head()

,month,W,L
0,2006-10-01,9.0,0.0
1,2006-11-01,19.0,2.0
2,2006-12-01,28.0,7.0
3,2007-01-01,32.0,12.0
4,2007-02-01,37.0,17.0


In [20]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green", legend="Wins")
p.line(source=source, x='month', y="L", color="red", legend="Loses")
p.title.text = "Anaheim Ducks Performance by Season"
p.title.text_font_size = "25px"
p.title.align = "center"
show(p)